# Loading External Dependencies

In [1]:
# Data Manipulation.
from pandas import read_csv, merge, DataFrame, concat

# Exploring Data

In [2]:
class Data:

    def __init__(self, paths : list[str]) -> None:

        # Preparing a container for storing the data.
        self.data = list()

        # Iterate over each path of data stored.
        for path in paths:
            # Add data one by one.
            self.data = self.load_data(path)

    def load_data(self, path : str) -> DataFrame:
        return read_csv(path)

In [3]:
pcd = Data(paths = [
    'Data/Purchasing Card Data 2014 v1.CSV',
    'Data/PCard Transactions 15-16.XSCORE',
    'Data/PCard 1617.csv',
    'Data/1718Pcard.csv'
])
pcd.data

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Purchasing Card Data 2014 v1.CSV'

## Loading the Data

In [ ]:
# Loading the data for 2014.
pcd_14 = read_csv('Data/Purchasing Card Data 2014 v1.CSV')
pcd_14

,Service Area,Account Description,Creditor,Transaction Date,JV Reference,JV Date,JV Value
0,Childrens Services,IT Services,123-REG.CO.UK,23/04/2014,93,20/05/2014,143.81
1,Childrens Services,Other Services,ACCESS EXPEDITIONS,03/04/2014,111,20/05/2014,"6,000.00"
2,Childrens Services,Equipment and Materials Repair,AFE SERVICELINE,02/04/2014,6,20/05/2014,309.38
3,Childrens Services,Equipment and Materials Repair,AFE SERVICELINE,02/04/2014,7,20/05/2014,218.76
4,Childrens Services,Building Repairs & Maintenance,ALLSOP & FRANCIS,15/04/2014,381,20/05/2014,306
...,...,...,...,...,...,...,...
4137,Adults and Communities,Postage,WWW.ROYALMAIL.COM,23/02/2015,5002,16/03/2015,124
4138,Children's Family Services,Equipment and Materials Purcha,WWW.SMYTHSTOYS.COM,20/02/2015,5188,16/03/2015,89.96
4139,Children's Family Services,Equipment and Materials Purcha,WWW.TTS-GROUPS.CO.U,06/02/2015,5207,16/03/2015,445.92
4140,Children's Family Services,Books-CDs-Audio-Video,WWW.WILDGOOSE.AC,09/02/2015,5271,16/03/2015,407.44


In [ ]:
# Loading the data for 2015-16.
pcd_15_16 = read_csv('Data/PCard Transactions 15-16.XSCORE')
pcd_15_16

,Service Area,Account Description,Creditor,Journal Date,Journal Reference,Total
0,Assurance,Miscellaneous Expenses,43033820 COSTA COFFEE,18/08/2015,5043.0,2
1,Children's Family Services,Miscellaneous Expenses,99 PLUS DISCOUNT MART,08/06/2015,4184.0,29.97
2,Children's Family Services,E19 - Learning Resources,99P STORES LTD,07/12/2015,6278.0,34.65
3,Children's Family Services,Equipment and Materials Purcha,99P STORES LTD,18/08/2015,5041.0,10.72
4,Children's Family Services,Subsistence,CHOPSTIX00000000000,21/05/2015,5750.0,33.7
...,...,...,...,...,...,...
3860,Children's Family Services,Food Costs,ZAHRA NEWSAGENT,17/11/2015,6042.0,3.5
3861,Children's Family Services,Food Costs,ZAHRA NEWSAGENT,20/01/2016,6751.0,4.5
3862,Children's Family Services,Food Costs,ZAHRA NEWSAGENT,21/03/2016,7535.0,4.5
3863,Children's Family Services,Food Costs,ZAHRA NEWSAGENT,31/03/2016,7639.0,4.5


In [ ]:
# Loading the data for 2016-17.
pcd_16_17 = read_csv('Data/PCard 1617.CSV')
pcd_16_17

,Service Area,Account Description,Creditor,Journal Date,Journal Reference,Total
0,Adults and Communities,Books-CDs-Audio-Video,AMAZON EU,05/12/2016,10510.0,45.00
1,Adults and Communities,Books-CDs-Audio-Video,AMAZON UK MARKETPLACE,05/12/2016,10509.0,426.57
2,Adults and Communities,Books-CDs-Audio-Video,AMAZON UK RETAIL AMAZO,06/12/2016,10524.0,121.38
3,Adults and Communities,Consumable Catering Supplies,WWW.ARGOS.CO.UK,01/03/2017,11667.0,78.94
4,Adults and Communities,CSG - IT,AMAZON UK MARKETPLACE,01/02/2017,10974.0,97.50
...,...,...,...,...,...,...
4577,Streetscene,Vehicle Running Costs,WWW.DVLA.GOV.UK,23/08/2016,9212.0,232.50
4578,Streetscene,Vehicle Running Costs,WWW.DVLA.GOV.UK,23/08/2016,9213.0,167.50
4579,Streetscene,Vehicle Running Costs,WWW.DVLA.GOV.UK,25/08/2016,9248.0,232.50
4580,Streetscene,Vehicle Running Costs,WWW.MOT-TESTING-CP,25/11/2016,10384.0,68.00


In [ ]:
# Loading the data for 2017-18.
pcd_17_18 = read_csv('Data/1718Pcard.CSV')
pcd_17_18

,FIN.TRANSACTION DATE,FIN.POSTING DATE,FIN.TRANSACTION AMOUNT,MCH.MERCHANT NAME,MCH.CITY NAME,FIN.ORIGINAL CURRENCY AMOUNT,FIN.ORIGINAL ISO CURRENCY CODE SYMBOL,FIN.INET CONVERSION
0,06/04/17,07/04/17,36.55,TESCO STORE 2296,COLNEY HATCH,36.55,GBP,1.0
1,06/04/17,07/04/17,58.75,AMFBOWLING.CO.UK,01442 840200,58.75,GBP,1.0
2,10/04/17,11/04/17,40.50,WWW.GOJUMPIN.COM,INTERNET,40.50,GBP,1.0
3,12/04/17,13/04/17,23.90,AMFBOWLING.CO.UK,01442 840200,23.90,GBP,1.0
4,12/04/17,13/04/17,24.28,VUE BSL LTD,LONDON,24.28,GBP,1.0
...,...,...,...,...,...,...,...,...
4926,16/03/18,19/03/18,20.00,GIFFGAFF,WWW.GIFFGAFF,20.00,GBP,1.0
4927,07/03/18,08/03/18,354.25,LOVE2SHOP.CO.UK,0345 717 1111,354.25,GBP,1.0
4928,21/03/18,22/03/18,51.88,AMAZON UK MARKETPLACE,800-279-6620,51.88,GBP,1.0
4929,22/03/18,26/03/18,10.00,WWW.BRENTGOV.UK,INTERNET,10.00,GBP,1.0


## Statistical 